# Poisonous Mushroom Classification

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.stats import chi2_contingency
# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
#Clustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
# Outlier Analysis
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
# Feature Selection
from sklearn.feature_selection import RFE, SelectKBest, mutual_info_classif
# Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Metrics
import time
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
D = pd.read_csv('secondary_data.csv', sep=';')

## Exploratory Data Analysis

In [ ]:
D.info()

In [ ]:
D.describe()

In [ ]:
# Compute Z-score Upper/Lower Bound outilers
num_data = D.describe()
z_score_outlier_threshold = 3
for column in num_data:
    mean = num_data[column]['mean']
    std = num_data[column]['std']
    ub = mean + z_score_outlier_threshold * std
    lb = mean - z_score_outlier_threshold * std
    
    num_outliers = len(D[(D[column] < lb) | (D[column] > ub)])
    
    if lb < 0:
        lb = 'N/A'
    print(f'{column} Z-score Outlier Count: {num_outliers}, Lower Bound: {lb}, Upper Bound: {ub}')
    
# Compute IQR Upper/Lower Bound outliers
num_columns = D[num_data.columns].copy()
Q1 = num_columns.quantile(.25)
Q3 = num_columns.quantile(.75)
IQR = Q3 - Q1
UB = Q3 + 1.5 * IQR
LB = Q1 - 1.5 * IQR
for column in num_columns:
    lb = LB[column]
    ub = UB[column]
    
    num_outliers = len(D[(D[column] < lb) | (D[column] > ub)])
    
    if lb < 0:
        lb = 'N/A'
    
    print(f'{column} IQR Outlier Count: {num_outliers}, Lower Bound: {lb}, Upper Bound: {ub}')

In [ ]:
# Plot histograms of each attribute
# Set size of plots for each histogram
fig, axs = plt.subplots(nrows=3, ncols=7, figsize=(28, 12))
axs = axs.flatten()

# Count number of missing data points for each feature
nan_count = D.isna().sum()

for i, column in enumerate(D.columns):
    # Plot the histogram
    sns.histplot(D[column], ax=axs[i], bins=50)
    
    # Display missing values
    missing_value_count = f"Missing value count: {nan_count[column]}"
    axs[i].text(.985, 0.985, missing_value_count, transform=axs[i].transAxes, ha="right", va="top", 
                 fontsize=10, color='red', bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.3'))
    
    # Set titles and labels
    axs[i].set_title(f'Histogram of {column}')
    axs[i].set_xlabel(column)
    axs[i].set_ylabel('Frequency')

# Display histograms
plt.tight_layout()
plt.show()

In [ ]:
# Plot boxplots of each numerical attribute
# Set size of plots for each boxplot
numerical_data = D[D.select_dtypes(include=['number']).columns]

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
axs = axs.flatten()

for i, column in enumerate(numerical_data.columns):
    # Plot the boxplot for the numeric column
    sns.boxplot(x=D[column], ax=axs[i])

    # Set titles and labels
    axs[i].set_title(f'Box Plot of {column}')
    axs[i].set_xlabel(column)
    axs[i].set_ylabel('Values')
    
# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Plot heatmaps to visualize relationships and identify correlations
# https://stackoverflow.com/questions/46498455/categorical-features-correlation
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    if n == 0:
        return np.nan
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    if min((kcorr-1), (rcorr-1)) == 0:
        return np.nan
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

num_corr_mat = numerical_data.corr()

plt.figure(figure=(8,6))
sns.heatmap(num_corr_mat, annot=True)

# Set title and labels
plt.title('Correlation Heatmap of Numerical Attributes')
plt.show()

categorical_data = D[D.select_dtypes(include=['object']).columns]
categorical_corr_mat = pd.DataFrame(np.zeros((len(categorical_data.columns), len(categorical_data.columns))),
                                columns=categorical_data.columns, index=categorical_data.columns)

for x in categorical_data.columns:
    for y in categorical_data.columns:
        if x == 1:
            categorical_corr_mat[x, y] = 1
        else:
            cat_confusion_matrix = pd.crosstab(categorical_data[x], categorical_data[y])
            categorical_corr_mat.loc[x, y] = cramers_v(cat_confusion_matrix.values)
plt.figure(figure=(20,20))
sns.heatmap(categorical_corr_mat)
plt.title('Correlation Heatmap of Categorical Attributes')
plt.show()

## Preprocessing

In [ ]:
# Remove features which are mostly NaN
drop_threshold = len(D) / 2
D_dropped_na = D.dropna(axis=1, thresh=drop_threshold)
print(D.drop(columns=D_dropped_na.columns))

# Split data into training(80%)/testing(20%) data
X = D_dropped_na.drop(columns='class')
y = D_dropped_na['class'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
# Get numerical/categorical feature mask
numerical_mask = X_train.select_dtypes(include=['number']).columns
categorical_mask = X_train.select_dtypes(include=['object']).columns

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        # Apply preprocessing steps to numerical features
        ('num', Pipeline(steps=[
                    # No missing numerical samples
                    # Apply Z-Score standardization
                    ('scaler', StandardScaler()),
                    # Only 3 numerical features, no need to apply PCA
                ]), numerical_mask),
        # Apply preprocessing steps to categorical features
        ('cat', Pipeline(steps=[
                    # Impute categorical data by mode
                    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                    # Encode categorical data using OneHotEncoding
                    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
                ]), categorical_mask),
    ])

In [ ]:
# Generate preprocessed data using preprocessing pipeline
preprocessing_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
    ])
X_train_prep = pd.DataFrame(preprocessing_pipeline.fit_transform(X_train))
X_test_prep = pd.DataFrame(preprocessing_pipeline.transform(X_test))

In [ ]:
# Retrieve column names
numerical_columns = numerical_mask
categorical_columns = preprocessing_pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_mask)
all_column_names = np.concatenate([numerical_columns, categorical_columns])
X_train_prep.columns = all_column_names
X_test_prep.columns = all_column_names

## Clustering

In [ ]:
# Evaluate clusters using various scores
def evaluate_clusters(X, labels):
    sil_score = silhouette_score(X, labels)
    ch_score = calinski_harabasz_score(X, labels)
    db_score = davies_bouldin_score(X, labels)
    return sil_score, ch_score, db_score

In [ ]:
# Visualize clusters
def visualize_clusters(X_pca, labels, algorithm):
    plt.figure(figsize=(8, 8))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels)
    plt.title(f"Clustering using {algorithm}")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.show()

In [ ]:
# Fit clustering algorithms on training data
X_cluster = X_train_prep.copy()

# Reduce dimensionality to visualize clusters
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster)

In [ ]:
# KMeans Clustering

# Find the best number of clusters for KMeans
def find_best_kmeans(X, min_clusters=2, max_clusters=10):
    best_n_clusters = min_clusters
    best_scores = {'silhouette': -1, 'calinski_harabasz': -1, 'davies_bouldin': float('inf')}
    best_labels = None

    for n_clusters in range(min_clusters, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=42).fit(X)
        labels = kmeans.labels_
        sil_score, ch_score, db_score = evaluate_clusters(X, labels)

        # Update if scores are better
        if sil_score > best_scores['silhouette']:
            best_scores = {'silhouette': sil_score, 'calinski_harabasz': ch_score, 'davies_bouldin': db_score}
            best_n_clusters = n_clusters
            best_labels = labels

    return best_n_clusters, best_scores, best_labels

best_n_clusters, best_scores, best_labels = find_best_kmeans(X_cluster)
print(f"Best KMeans Clusters: {best_n_clusters}")
print(f"Silhouette Score: {best_scores['silhouette']:.4f}")
print(f"Calinski Harabasz Score: {best_scores['calinski_harabasz']:.4f}")
print(f"Davies Bouldin Score: {best_scores['davies_bouldin']:.4f}")

visualize_clusters(X_pca, best_labels, f'KMeans (n_clusters={best_n_clusters})')

In [ ]:
dbscan = DBSCAN(eps=3, min_samples=2).fit(X_cluster)
sil_score, ch_score, db_score = evaluate_clusters(X_cluster, dbscan.labels_)
print("\nDBSCAN Clustering:")
print(f"Silhouette Score: {sil_score:.4f}")
print(f"Calinski Harabasz Score: {ch_score:.4f}")
print(f"Davies Bouldin Score: {db_score:.4f}")
visualize_clusters(X_pca, dbscan.labels_, 'DBSCAN')

In [ ]:
# Evaluate clusters using various scores
def evaluate_clusters(X, labels):
    sil_score = silhouette_score(X, labels)
    ch_score = calinski_harabasz_score(X, labels)
    db_score = davies_bouldin_score(X, labels)
    print('Silhouette Score: %1.4f' % sil_score)
    print('Calinski Harabasz Score: %1.4f' % ch_score)
    print('Davies Bouldin Score: %1.4f' % db_score)

# Agglomerative clustering can only handle small datasets
# Sample from training data
X_agg_sample = X_cluster.sample(n=5000)
agg = AgglomerativeClustering().fit(X_agg_sample)
evaluate_clusters(X_agg_sample, agg.labels_)
X_agg_pca = pca.transform(X_agg_sample)
visualize_clusters(X_agg_pca, agg.labels_, 'Agglomerative Clustering')

# Visualize Dendrogram for Agglomerative Clustering
# https://www.geeksforgeeks.org/hierarchical-clustering-with-scikit-learn/
Z = linkage(X_agg_pca, method='ward')
plt.figure(figsize=(10, 7))
dendrogram(Z)
plt.title('Dendrogram')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()

## Outlier Analysis

In [ ]:
# Visualize Outliers
def visualize_outliers(X_pca, outliers, model):
    X_outliers = X_pca[outliers == -1]
    X_inliers = X_pca[outliers == 1]
    x_out, y_out = X_outliers[:, 0], X_outliers[:, 1]
    x_in, y_in = X_inliers[:, 0], X_inliers[:, 1]
    
    # Plot the outliers and inliers
    plt.figure(figsize=(6, 6))
    plt.scatter(x_in, y_in, label='Inliers', color='blue')
    plt.scatter(x_out, y_out, label='Outliers', color='red')
    plt.title(f'Outliers Detected by {model}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.show()

In [ ]:
# Fit Outlier Detection algorithms on train dataset
X_outliers = X_train_prep.copy()

# Reduce dimensionality to visualize clusters
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_outliers)

In [ ]:
# Isolation Forest
iso_outliers = IsolationForest(random_state=0).fit_predict(X_outliers)
visualize_outliers(X_pca, iso_outliers, 'Isolation Forest')

In [ ]:
# LocalOutlierFactor
lof_outliers = LocalOutlierFactor().fit_predict(X_outliers)
visualize_outliers(X_pca, lof_outliers, 'LocalOutlierFactor')

In [ ]:
# EllipticEnvelope
env_outliers = EllipticEnvelope(random_state=0).fit_predict(X_outliers)
visualize_outliers(X_pca, env_outliers, 'EllipticEnvelope')

# Classification Helper Functions

In [ ]:
def evaluate_model_consistency(X_train, y_train, model):
    # Perform cross-validation to evaluate model consistency
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    print("Mean Cross-validation Scores: ", np.mean(cv_scores))

In [ ]:
# Only RandomForest, GaussianNB, and LogisticRegression imported
def train_model(X_train, X_test, y_train, y_test, model):
    # Measure time to train/predict
    start_time = time.time()
    
    # Fit model and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    end_time = time.time()
    model_time = end_time - start_time
    
    # Get class probability for roc_auc_score/roc_curve
    y_prob = model.predict_proba(X_test)[:, 1]
    
    return y_pred, y_prob, model_time

In [ ]:
def evaluate_model(y_test, y_pred, y_prob, runtime):
    # Output various scores to evaluate model
    print("Model train and predict time: %1.4f" % runtime)
    print(classification_report(y_test, y_pred))
    print("AUC-ROC Score: %1.4f" % roc_auc_score(y_test, y_prob))

In [ ]:
def visualize_results(y_test, y_pred, y_prob, model):
    # Plot confusion matrix
    labels = ['e', 'p']
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='g', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model} Confusion Matrix')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.show()
    
    # Plot ROC Curve
    # https://stackoverflow.com/questions/25009284/how-to-plot-roc-curve-in-python
    fpr, tpr, thresholds = roc_curve(y_test, y_prob, pos_label='p')
    roc_auc = roc_auc_score(y_test, y_prob)
    plt.figure(figsize=(6, 6))
    plt.title(f'{model} Receiver Operating Characteristic')
    plt.plot(fpr, tpr, label=f'AUC = %1.4f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
# Run the full pipeline for a model
def train_and_evaluate_model(X_train, X_test, y_train, y_test, model):
    evaluate_model_consistency(X_train, y_train, model)
    y_pred, y_prob, runtime = train_model(X_train, X_test, y_train, y_test, model)
    evaluate_model(y_test, y_pred, y_prob, runtime)
    visualize_results(y_test, y_pred, y_prob, model)

## Feature Selection

In [ ]:
# View feature importance using LogisticRegression coefficients
def visualize_feature_importance(coeffs, X, model):
    importance_df = pd.DataFrame()
    importance_df['Feature'] = X.columns
    importance_df['Coefficient'] = np.abs(coeffs)

    importance_df = importance_df.sort_values(by='Coefficient', ascending=False)
    plt.barh(importance_df['Feature'], importance_df['Coefficient'])
    plt.xlabel('Importance as Absolute Value of Coefficients')
    plt.title(f'Feature Importance of LogisticRegression using {model}')
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
# Evaluate classification model without Feature Selection
# Use LogisticRegression as model
X_train_fe = X_train_prep.copy()
X_test_fe = X_test_prep.copy()
clf = LogisticRegression(max_iter=10000, random_state=0)
train_and_evaluate_model(X_train_fe, X_test_fe, y_train, y_test, clf)

In [ ]:
# Fit train dataset on RecursiveFeatureSelection
rfe = RFE(clf, n_features_to_select=25).fit(X_train_fe, y_train)
X_train_rfe = X_train_fe.loc[:, rfe.support_]
X_test_rfe = X_test_fe.loc[:, rfe.support_]

In [ ]:
# View selected columns from RFE
X_train_rfe.columns

In [ ]:
# Evaluate classification model with Feature Selection
train_and_evaluate_model(X_train_rfe, X_test_rfe, y_train, y_test, clf)

In [ ]:
# Visualize feature importance after RFE
coeffs = clf.coef_[0]
visualize_feature_importance(coeffs, X_train_rfe, 'Recursive Feature Elimination')

In [ ]:
# Select the k best features based on MutualInformation
k_best = SelectKBest(mutual_info_classif, k=25)
X_train_mi = k_best.fit(X_train_fe, y_train)

In [ ]:
# Transform train and test data on selected features
k_best_support = X_train_fe.columns[k_best.get_support(indices=True)]
X_train_mi = X_train_fe.loc[:, k_best_support]
X_test_mi = X_test_fe.loc[:, k_best_support]
k_best_support

In [ ]:
# Evaluate classification model with mutual information
train_and_evaluate_model(X_train_mi, X_test_mi, y_train, y_test, clf)

In [ ]:
# Visualize feature importance after mutual information
coeffs = clf.coef_[0]
visualize_feature_importance(coeffs, X_train_mi, 'Mutual Information')

## Classification

In [ ]:
# Train and Evaluate classification models
# Using GaussianNB, RandomForests, and LogisticRegression
X_train_classify = X_train_rfe.copy()
X_test_classify = X_test_rfe.copy()

In [ ]:
# GaussianNB
gnb = GaussianNB()
train_and_evaluate_model(X_train_classify, X_test_classify, y_train, y_test, gnb)

In [ ]:
# RandomForestClassified
rfc = RandomForestClassifier(random_state=0)
train_and_evaluate_model(X_train_classify, X_test_classify, y_train, y_test, rfc)

In [ ]:
# LogisticRegression
logr = LogisticRegression(max_iter=10000, random_state=0)
train_and_evaluate_model(X_train_classify, X_test_classify, y_train, y_test, logr)

## Hyperparameter Tuning

In [ ]:
# Perform hyperparameter tuning using Grid Search on LogisticRegrssion
X_train_hyper = X_train_rfe.copy()
X_test_hyper = X_test_rfe.copy()
logr = LogisticRegression(max_iter=10000, random_state=0)

# Define grid search parameters
param_grid = {
    'C': [.0001, .001, .01, .1, 1, 10],
    'tol': [.00001, .0001, .001, .01 , .1, 1]
}

In [ ]:
# Fit the training data on GridSearchCV
grid_search = GridSearchCV(estimator=logr, param_grid=param_grid, scoring='accuracy', verbose=2)
grid_search.fit(X_train_hyper, y_train)

In [ ]:
# Visualize Results from GridSearch
gs_results = grid_search.cv_results_

for tol in param_grid['tol']:
    tol_mask = gs_results['param_tol'] == tol
    plt.plot(gs_results['param_C'][tol_mask], gs_results['mean_test_score'][tol_mask], label=tol, marker='o')
    
plt.xscale('log')
plt.title('Accuracy vs C Accross Different Tolerence')
plt.xlabel('C (Inverse of regularization strength)')
plt.ylabel('Mean Cross-Validation Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluate model using best params from GridSearch
C = grid_search.best_params_['C']
tol = grid_search.best_params_['tol']
print(f"Optimal C: {C}")
print(f"Optimal tol: {tol}")

logr_gs = LogisticRegression(C=C, tol=tol, max_iter=10000, random_state=0)
train_and_evaluate_model(X_train_hyper, X_test_hyper, y_train, y_test, logr_gs)

In [ ]:
# Perform hyperparameter tuning using RandomSearch on RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)


# Define Random Search parameters
param_dists = {
    'n_estimators': [50, 100, 250, 500],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [2, 5, 10, 25, 50, 100, None],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
}

In [ ]:
# Fit the training data on RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rfc, param_distributions=param_dists, n_iter=50, scoring='accuracy', verbose=2, random_state=0)
random_search.fit(X_train_hyper, y_train)

In [ ]:
# Evaluate model using best params from RandomizedSerach
n_estimators = random_search.best_params_['n_estimators']
min_samples_split = random_search.best_params_['min_samples_split']
min_samples_leaf = random_search.best_params_['min_samples_leaf']
max_features = random_search.best_params_['max_features']
max_depth = random_search.best_params_['max_depth']
criterion = random_search.best_params_['criterion']

for param in random_search.best_params_:
    print(f"Optimal {param}: {random_search.best_params_[param]}")

rfc_rs = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split
                                , min_samples_leaf=min_samples_leaf, max_features=max_features,
                                max_depth=max_depth, criterion=criterion, random_state=0)
train_and_evaluate_model(X_train_hyper, X_test_hyper, y_train, y_test, rfc_rs)